In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

In [ ]:
def clear_trash():
  for a_file in my_drive.ListFile({'q': "trashed = true"}).GetList():
    # print the name of the file being deleted.
    try:
      print(f"the file {a_file['title']}, is about to get deleted permanently.")
      # delete the file permanently.
      a_file.Delete()
    except:
      pass

In [ ]:
# clear_trash()

In [ ]:
# !cp /content/drive/MyDrive/TalkLikeMom/conversations-main.zip /content
# !unzip conversations-main.zip

In [ ]:
cd /content/conversations-main/Classifier/

/content/conversations-main/Classifier


In [ ]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch
torch.cuda.is_available()

True

In [ ]:
# !pip uninstall --yes torchaudio torchdata torchtext torchvision

In [ ]:
# !pip install -q torch==1.13.0

In [ ]:
torch.__version__

'2.1.0+cu118'

In [ ]:
import sys
sys.path.append('.')

In [ ]:
# cp ../utils.py .

In [ ]:
from utils import load_jsonl, dump_jsonl, set_random_seed

In [ ]:
from data_loader import get_task1_conver, get_task2_conver, preprocess

In [ ]:
import pandas as pd

In [ ]:
# df = get_task1_conver("../Task1//annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [ ]:

# df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [ ]:
# df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = [], only_user=False)
# # print(df[0]["text"][0])
# pd.concat(df).groupby("label").count()

In [ ]:
# "DONE"

In [ ]:
# !pip install -q transformers pythainlp datasets evaluate sentencepiece
# # !pip install -q accelerate -U
# !pip install -q transformers[torch]
# # !pip install -q
# !pip install -q shap nlp

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import wandb
# from sklearn.metrics import precision_recall_fscore_support, classification_report
# from pythainlp.tokenize import word_tokenize
import torch
import datasets
from datasets import Dataset, DatasetDict
from transformers import DataCollatorWithPadding
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer



In [ ]:
import random
def set_random_seed(seed=42):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    np.random.RandomState(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) #seed all gpus
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False

In [ ]:
from sklearn.utils import compute_class_weight
import torch.nn as nn
import os, shutil
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import mean_squared_error, r2_score

def run_exp(out_dir, df, seed=42, report="none", regressor_configs=None):

    set_random_seed(seed)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    # device = torch.device("cpu")
    print("START")
    print("step 1: load data")
    train, val, test = df

#     train = train.head(100)
#     val = val.head(100)
#     test = test.head(100)

    print("step 2: load tokenizer")
    model_name = "airesearch/wangchanberta-base-att-spm-uncased"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    num_added_toks = tokenizer.add_special_tokens({"additional_special_tokens": ["usr", "sys", "rep"]})

    print("step 3: init data")
    ds = DatasetDict()
    ds['train'] = Dataset.from_pandas(train)
    ds['val'] = Dataset.from_pandas(val)
    ds['test'] = Dataset.from_pandas(test)

    if regressor_configs is None:
        labels = train["label"].unique()
        num_labels = len(labels)
        print(labels)

        class_weights = compute_class_weight("balanced", classes=labels, y=train["label"].values)
        class_weights = torch.tensor(class_weights).float().to(device)

        id2label = {i:l for i, l in enumerate(labels)}
        label2id = {l:i for i, l in enumerate(labels)}

        def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
            texts = [preprocess(t) for t in d["text"]]
    #         print(texts)
            tokens = tokenizer(texts, truncation=True, max_length=max_length)
            num = [len(t) for t in tokens["input_ids"]]
    #         print(num)
    #         print("AVG", len(num), sum(num)/len(num))
            tokens["label"] = [label2id[label] for label in d["label"]]
            return tokens
    else:
#         labels = train["label"].unique()
        num_labels = 1

        id2label = {1: regressor_configs["label"]}
        label2id = [regressor_configs["label"]]

        def word_tokenize(d, tokenizer=None, label2id=None, max_length=256):
            texts = [preprocess(t) for t in d["text"]]
    #         print(texts)
            tokens = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length)
            # num = [len(t) for t in tokens["input_ids"]]
    #         print(num)
    #         print("AVG", len(num), sum(num)/len(num))
            tokens["label"] = [regressor_configs["label_fn"](label) for label in d["label"]]
            return tokens



    tokenized_ds = ds.map(word_tokenize, batched=True, fn_kwargs={"tokenizer":tokenizer, "label2id": label2id, "max_length":max_length})
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

    print("step 4: load model")
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id);
    model.resize_token_embeddings(len(tokenizer)+1)
    model = model.to(device)

    # print(model)

    if regressor_configs is None:
        metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            return metrics.compute(predictions=predictions, references=labels, average="macro")
    else:
        label_fn = regressor_configs["label_fn"]
        def compute_metrics(eval_pred):
            predictions, actual = eval_pred
            predictions = predictions.reshape(-1)

            predicted_labels = [label_fn(p) for p in predictions]
            actual_labels = [label_fn(p) for p in actual]
            p, r, f1, _ = precision_recall_fscore_support(actual_labels, predicted_labels, average='macro')

            return {
                "r2_score": r2_score(actual, predictions),
                "mean_squared_error": np.sqrt(mean_squared_error(actual, predictions)),
                "accuracy": accuracy_score(actual_labels, predicted_labels),
                "f1": f1,
                "precision": p,
                "recall": r,
            }


    print("step 5: fine-tune")

    training_args = TrainingArguments(
        output_dir=out_dir,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to=report,
        metric_for_best_model="f1",
        save_total_limit=2,
        load_best_model_at_end=True,
        push_to_hub=False,
        run_name=out_dir,
    )

    # print(training_args)

    if regressor_configs is None:
        class CustomTrainer(Trainer):
            def compute_loss(self, model, inputs, return_outputs=False):
                labels = inputs.get("labels")
                # forward pass
                outputs = model(**inputs)

                logits = outputs.get("logits")

                loss_fct = nn.CrossEntropyLoss(weight=class_weights)
                loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
                return (loss, outputs) if return_outputs else loss
    else:
        CustomTrainer = Trainer

    trainer = CustomTrainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_ds["train"],
        eval_dataset=tokenized_ds["val"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    best_ckpt_path = trainer.state.best_model_checkpoint
    print(best_ckpt_path)

    modle_out_path = out_dir+"/best_model"
    if os.path.exists(modle_out_path):
        shutil.rmtree(modle_out_path)

    os.rename(best_ckpt_path, modle_out_path)
    best_ckpt_path = modle_out_path

    train.to_csv(modle_out_path+"/train.csv", index=False)
    val.to_csv(modle_out_path+"/val.csv", index=False)
    test.to_csv(modle_out_path+"/test.csv", index=False)

    print("step 6: evaluate")
    eval_test = trainer.evaluate(tokenized_ds["test"])
    eval_val = trainer.evaluate(tokenized_ds["val"])
    print(eval_test)

    print("DONE")

    return trainer, eval_val, eval_test

In [ ]:
# import IPython
# display(IPython.display.Audio(url="/content/sound.mp3", autoplay=False))

## Task1: Train Model

In [ ]:
report = "none"
batch_size = 16
max_length = 128
num_epochs = 20

In [ ]:
# import os
# stream = os.popen('nohup python3 run_train_task_classifier.py > train2.out &')
# output = stream.read()
# output

In [ ]:
# df[0]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [ ]:
# df[1]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [ ]:
# df[2]["label"].value_counts().loc[['1. Close', '2. Know each other', "3. Don't know each other"]].plot.bar()

In [ ]:
def closeness_label_fn(label):
    if label == '1. Close':
        return 1
    elif label =='2. Know each other':
        return 0.5
    elif label == "3. Don't know each other":
        return 0
    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.66:
        return '1. Close'
    elif label > 0.33:
        return '2. Know each other'
    else:
        return "3. Don't know each other"

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/task1_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1096 60 60
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1096 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

step 4: load model


...

step 5: fine-tune


You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.113107,0.463349,0.336313,0.700000,0.651453,0.664868,0.667069
2,No log,0.097952,0.535254,0.312972,0.733333,0.664360,0.668254,0.668334
3,No log,0.107441,0.490228,0.327783,0.616667,0.517028,0.538889,0.504428
4,No log,0.108551,0.484964,0.329471,0.666667,0.597436,0.611172,0.595583
5,No log,0.105157,0.501066,0.324280,0.733333,0.681808,0.688845,0.698643
6,No log,0.096287,0.543153,0.310301,0.683333,0.597513,0.629135,0.597136
7,No log,0.084581,0.598691,0.290829,0.766667,0.693105,0.697074,0.694732
8,0.094200,0.090379,0.571184,0.300631,0.750000,0.662148,0.665955,0.660283
9,0.094200,0.104343,0.504930,0.323021,0.733333,0.646291,0.656310,0.643202
10,0.094200,0.099622,0.527328,0.315630,0.700000,0.612711,0.637407,0.611629


./Regressors/task1_clse_usr4/checkpoint-483
step 6: evaluate


{'eval_loss': 0.0672571137547493, 'eval_r2_score': 0.6704653551822286, 'eval_mean_squared_error': 0.2593397796154022, 'eval_accuracy': 0.7166666666666667, 'eval_f1': 0.6195600392321704, 'eval_precision': 0.625518341307815, 'eval_recall': 0.6154719235364396, 'eval_runtime': 0.1522, 'eval_samples_per_second': 394.159, 'eval_steps_per_second': 26.277, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.0845814049243927, 'eval_r2_score': 0.5986911582486485, 'eval_mean_squared_error': 0.2908288538455963, 'eval_accuracy': 0.7666666666666667, 'eval_f1': 0.6931048551611587, 'eval_precision': 0.6970741901776384, 'eval_recall': 0.6947319990798251, 'eval_runtime': 0.1514, 'eval_samples_per_second': 396.215, 'eval_steps_per_second': 26.414, 'epoch': 20.0}
Result[test] {'eval_loss': 0.0672571137547493, 'eval_r2_score': 0.6704653551822286, 'eval_mean_squared_error': 0.2593397796154022, 'eval_accuracy': 0.7166666666666667, 'eval_f1': 0.6195600392321704, 'eval_precision': 0.6255183413078

In [ ]:
# ls Regressors/task1_clse_usr0/best_model

In [ ]:
# !zip -r task1.zip Regressors/task1_*

In [ ]:
# !cp -r Regressors/task1_* /content/drive/MyDrive/TalkLikeMom/src/Classifier/Regressors

In [ ]:
# !mkdir ShapleyValuesV3

In [ ]:
import shap
import scipy as sp

import pickle

def load_shap_values(filepath):
    with open(filepath, 'rb') as fin:
        obj = pickle.load(fin)
    return obj

def save_shap_values(filepath, obj):
    with open(filepath, 'wb') as fin:
        pickle.dump(obj, fin)


def get_shapley(df, trainer, shap_path):
    train, val, test = df
    train["split"] = "train"
    val["split"] = "val"
    test["split"] = "test"

    df = pd.concat([train, test, val])
    print("DATA SIZE", len(df))
    def f(x):
        tokenizer = trainer.tokenizer
        model = trainer.model

        # print(type(x.tolist()))
        tv = tokenizer(x.tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors="pt")
        input_ids = tv["input_ids"].cuda()
        attention_mask = tv["attention_mask"].cuda()
        # tv = tokenizer(x, truncation=True, max_length=max_length).cuda()
        outputs = model(input_ids, attention_mask=attention_mask)[0].detach().cpu().numpy()

        # scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
        # val = sp.special.logit(scores[:,1]) # use one vs rest logit units

        val = outputs.reshape(-1)
        return val

    explainer = shap.Explainer(f, trainer.tokenizer)
    data = {
        "label": df["label"].values,
        "text": [preprocess(t) for t in df["text"].values],
    }

    shap_values = explainer(data)

    save_shap_values(shap_path, {
        "values": shap_values,
        "inputs": data
    })

    return shap_values


In [ ]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task1_clse_regressor.pkl");

DATA SIZE 1216


PartitionExplainer explainer: 1217it [32:19,  1.60s/it]


In [ ]:
# mkdir /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3

In [ ]:
cp ./ShapleyValuesV3/task1_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task1_clse_regressor.pkl

In [ ]:
def authority1_label_fn(label):
    if label == '0. Very respect':
        return 1
    elif label =='1. Respect':
        return 0.5
    elif label == "2. Normal":
        return 0
    elif type(label)==str:
        assert(False)

    # [0, 0.33) =>
    # [0.33, 0.66) =>
    # [0.66, 1] =>

    if label > 0.66:
        return '0. Very respect'
    elif label > 0.33:
        return '1. Respect'
    else:
        return "2. Normal"

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task1/annotated_conersations.jsonl", "authority", skips = ["3. Not respect"], only_user=True)
  trainer, eval_val, eval_test = run_exp(f"./Regressors/task1_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority1_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1234 records from ../Task1/annotated_conersations.jsonl
N 1098 61 61
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1098 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.148356,0.049038,0.385170,0.475410,0.333903,0.314695,0.383660
2,No log,0.162765,-0.043322,0.403441,0.540984,0.297314,0.267273,0.348366
3,No log,0.131895,0.154552,0.363174,0.459016,0.336111,0.334499,0.386275
4,No log,0.156230,-0.001431,0.395259,0.491803,0.515152,0.622727,0.593791
5,No log,0.133500,0.144268,0.365376,0.606557,0.545212,0.591165,0.527124
6,No log,0.125238,0.197223,0.353890,0.590164,0.454026,0.661698,0.450980
7,No log,0.122465,0.215003,0.349949,0.540984,0.520056,0.544136,0.518301
8,0.151200,0.150335,0.036352,0.387731,0.475410,0.476358,0.510053,0.509477
9,0.151200,0.123056,0.211212,0.350793,0.606557,0.575403,0.647796,0.564379
10,0.151200,0.122770,0.213043,0.350386,0.622951,0.570731,0.641642,0.549346


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.


./Regressors/task1_auth_usr4/checkpoint-1173
step 6: evaluate


{'eval_loss': 0.15073907375335693, 'eval_r2_score': 0.12971296335618143, 'eval_mean_squared_error': 0.38825127482414246, 'eval_accuracy': 0.5737704918032787, 'eval_f1': 0.5174270007853217, 'eval_precision': 0.5716374269005847, 'eval_recall': 0.5023809523809524, 'eval_runtime': 0.1513, 'eval_samples_per_second': 403.051, 'eval_steps_per_second': 26.43, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.15073907375335693, 'eval_r2_score': 0.12971296335618143, 'eval_mean_squared_error': 0.38825127482414246, 'eval_accuracy': 0.5737704918032787, 'eval_f1': 0.5174270007853217, 'eval_precision': 0.5716374269005847, 'eval_recall': 0.5023809523809524, 'eval_runtime': 0.1513, 'eval_samples_per_second': 403.051, 'eval_steps_per_second': 26.43, 'epoch': 20.0}
Result[test] {'eval_loss': 0.141339972615242, 'eval_r2_score': 0.09401222960004796, 'eval_mean_squared_error': 0.3759520351886749, 'eval_accuracy': 0.639344262295082, 'eval_f1': 0.6030561178587495, 'eval_precision': 0.646428571

In [ ]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task1_auth_regressor.pkl");

DATA SIZE 1220


PartitionExplainer explainer: 1221it [32:28,  1.61s/it]


In [ ]:
cp ./ShapleyValuesV3/task1_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task1_auth_regressor.pkl

In [ ]:
"DONEONEOEN"

'DONEONEOEN'

## Task2: Train Model

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=i, skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1495 186 186
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1495 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

Map:   0%|          | 0/186 [00:00<?, ? examples/s]

step 4: load model


...

step 5: fine-tune


You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.099166,0.141886,0.314906,0.725806,0.417336,0.695690,0.454503
2,No log,0.088076,0.237853,0.296775,0.704301,0.469121,0.580972,0.483754
3,No log,0.085125,0.263383,0.291762,0.741935,0.404652,0.662971,0.402525
4,No log,0.086494,0.251541,0.294098,0.752688,0.533418,0.586357,0.528746
5,No log,0.087163,0.245752,0.295233,0.731183,0.454854,0.538069,0.436406
6,0.106300,0.088542,0.233817,0.297560,0.779570,0.547146,0.600412,0.524453
7,0.106300,0.091429,0.208838,0.302372,0.747312,0.455669,0.537292,0.444865
8,0.106300,0.106650,0.077126,0.326572,0.784946,0.469539,0.625586,0.446717
9,0.106300,0.086933,0.247744,0.294843,0.774194,0.487649,0.553020,0.469276
10,0.106300,0.095725,0.171657,0.309395,0.774194,0.479139,0.562296,0.453409


./Regressors/task2_clse_usr4/checkpoint-1692
step 6: evaluate


{'eval_loss': 0.09616263210773468, 'eval_r2_score': 0.13052147546341697, 'eval_mean_squared_error': 0.3101010024547577, 'eval_accuracy': 0.7795698924731183, 'eval_f1': 0.4922991480963573, 'eval_precision': 0.548528965726418, 'eval_recall': 0.48519317636964693, 'eval_runtime': 0.4463, 'eval_samples_per_second': 416.729, 'eval_steps_per_second': 26.886, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.09616263210773468, 'eval_r2_score': 0.13052147546341697, 'eval_mean_squared_error': 0.3101010024547577, 'eval_accuracy': 0.7795698924731183, 'eval_f1': 0.4922991480963573, 'eval_precision': 0.548528965726418, 'eval_recall': 0.48519317636964693, 'eval_runtime': 0.4463, 'eval_samples_per_second': 416.729, 'eval_steps_per_second': 26.886, 'epoch': 20.0}
Result[test] {'eval_loss': 0.08269355446100235, 'eval_r2_score': 0.2844256998109531, 'eval_mean_squared_error': 0.28756484389305115, 'eval_accuracy': 0.7849462365591398, 'eval_f1': 0.5544791506487364, 'eval_precision': 0.623376

In [ ]:
# report = "none"
# batch_size = 16
# max_length = 128
# num_epochs = 100

In [ ]:
# df = get_task2_conver("../Task2/annotated/annotated.jsonl", "closeness", seed=42, skips = ["4. Don't like each other"], only_user=True)
# trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_clse_usr{i}", df, seed=42, report=report, regressor_configs={
#     "label": "close",
#     "not_label": "not_close",
#     "label_fn": closeness_label_fn,
# })

# print("Result[val]", eval_val)
# print("Result[test]", eval_test)

In [ ]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task2_clse_regressor.pkl");

DATA SIZE 1867


PartitionExplainer explainer: 1868it [34:33,  1.12s/it]


In [ ]:
cp ./ShapleyValuesV3/task2_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task2_clse_regressor.pkl

In [ ]:
def authority2_label_fn(label):
    if label == '1. Respect':
        return 1
    elif label =='2. Normal':
        return 0.5
    elif label == "3. Not respect":
        return 0
    elif type(label)==str:
        assert(False)

    if label > 0.66:
        return '1. Respect'
    elif label > 0.33:
        return '2. Normal'
    else:
        return "3. Not respect"

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task2_conver("../Task2/annotated/annotated.jsonl", "authority", seed=i, skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task2_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 2486 records from ../Task2/annotated/annotated.jsonl
N 1876 234 234
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1876 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

Map:   0%|          | 0/234 [00:00<?, ? examples/s]

step 4: load model


...

step 5: fine-tune


You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.063150,0.244889,0.251297,0.636752,0.550702,0.663004,0.612144
2,No log,0.041020,0.509507,0.202534,0.713675,0.666321,0.648641,0.690393
3,No log,0.051595,0.383053,0.227146,0.628205,0.629004,0.632007,0.680199
4,No log,0.045253,0.458887,0.212728,0.730769,0.689272,0.685745,0.696141
5,0.064700,0.044835,0.463888,0.211743,0.764957,0.708215,0.713150,0.719055
6,0.064700,0.056398,0.325629,0.237482,0.709402,0.663823,0.700006,0.653586
7,0.064700,0.054352,0.350091,0.233135,0.726496,0.655191,0.693439,0.640578
8,0.064700,0.050003,0.402098,0.223613,0.747863,0.686360,0.725291,0.667232
9,0.045200,0.064368,0.230328,0.253708,0.717949,0.632169,0.657517,0.613653
10,0.045200,0.048781,0.416701,0.220865,0.743590,0.670440,0.688287,0.663454


./Regressors/task2_auth_usr4/checkpoint-1770
step 6: evaluate


{'eval_loss': 0.03620758280158043, 'eval_r2_score': 0.5651277847174239, 'eval_mean_squared_error': 0.1902828961610794, 'eval_accuracy': 0.8034188034188035, 'eval_f1': 0.7663492063492064, 'eval_precision': 0.750356213462349, 'eval_recall': 0.789275673422015, 'eval_runtime': 0.5613, 'eval_samples_per_second': 416.887, 'eval_steps_per_second': 26.724, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.03620758280158043, 'eval_r2_score': 0.5651277847174239, 'eval_mean_squared_error': 0.1902828961610794, 'eval_accuracy': 0.8034188034188035, 'eval_f1': 0.7663492063492064, 'eval_precision': 0.750356213462349, 'eval_recall': 0.789275673422015, 'eval_runtime': 0.5613, 'eval_samples_per_second': 416.887, 'eval_steps_per_second': 26.724, 'epoch': 20.0}
Result[test] {'eval_loss': 0.038442786782979965, 'eval_r2_score': 0.5403236057433978, 'eval_mean_squared_error': 0.19606831669807434, 'eval_accuracy': 0.7991452991452992, 'eval_f1': 0.7608664695098214, 'eval_precision': 0.75526745786

In [ ]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task2_auth_regressor.pkl");

In [ ]:
cp ./ShapleyValuesV3/task2_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task2_auth_regressor.pkl

## Task3: Train Model

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "closeness", skips = ["4. Don't like each other"], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task3_clse_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "close",
      "not_label": "not_close",
      "label_fn": closeness_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

In [ ]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task3_clse_regressor.pkl");

DATA SIZE 1210


PartitionExplainer explainer: 1211it [32:00,  1.60s/it]


In [ ]:
cp ./ShapleyValuesV3/task3_clse_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task3_clse_regressor.pkl


In [ ]:
,

In [ ]:
best_trainer = None
best_eval_val = None
best_eval_test = None

for i in range(5):
  df = get_task1_conver("../Task3/annotated/annotated.jsonl", "authority", skips = [], only_user=True)
  trainer, eval_test, eval_val = run_exp(f"./Regressors/task3_auth_usr{i}", df, seed=i, report=report, regressor_configs={
      "label": "respect",
      "not_label": "not_respect",
      "label_fn": authority2_label_fn,
  })

  print(f"==== round {i+1} ====")
  print("Result[val]", eval_val)
  print("Result[test]", eval_test)

  if (best_eval_val is None) or (best_eval_val["eval_f1"] < eval_val["eval_f1"]):
    best_eval_val = eval_val
    best_eval_test = eval_test
    best_trainer = trainer

  del trainer

print()
print()
print("===================================")
print()
print("BEST MODEL")
print("Result[val]", best_eval_val)
print("Result[test]", best_eval_test)

Loaded 1221 records from ../Task3/annotated/annotated.jsonl
N 1099 61 61
cuda
START
step 1: load data
step 2: load tokenizer
step 3: init data


Map:   0%|          | 0/1099 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

step 4: load model


...

You're using a CamembertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,R2 Score,Mean Squared Error,Accuracy,F1,Precision,Recall
1,No log,0.071531,-0.756870,0.267452,0.721311,0.439194,0.463768,0.490196
2,No log,0.055806,-0.370657,0.236233,0.786885,0.450196,0.420532,0.509804
3,No log,0.062547,-0.536227,0.250095,0.819672,0.623810,0.639456,0.627451
4,No log,0.047592,-0.168922,0.218157,0.803279,0.611772,0.627778,0.620915
5,No log,0.063670,-0.563811,0.252330,0.655738,0.472934,0.532479,0.534314
6,No log,0.066106,-0.623637,0.257111,0.590164,0.440310,0.517460,0.508170
7,No log,0.039881,0.020489,0.199701,0.819672,0.623810,0.639456,0.627451
8,0.043400,0.043395,-0.065825,0.208314,0.754098,0.481111,0.525850,0.475490
9,0.043400,0.049500,-0.215777,0.222486,0.803279,0.611772,0.627778,0.620915
10,0.043400,0.039488,0.030143,0.198715,0.770492,0.590247,0.609903,0.607843


Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted s

./Regressors/task3_auth_usr4/checkpoint-207
step 6: evaluate


{'eval_loss': 0.025045862421393394, 'eval_r2_score': 0.07267999970304528, 'eval_mean_squared_error': 0.158258855342865, 'eval_accuracy': 0.9180327868852459, 'eval_f1': 0.8331943286071727, 'eval_precision': 0.8484848484848485, 'eval_recall': 0.8209876543209876, 'eval_runtime': 0.1544, 'eval_samples_per_second': 395.046, 'eval_steps_per_second': 25.905, 'epoch': 20.0}
DONE
==== round 5 ====
Result[val] {'eval_loss': 0.025045862421393394, 'eval_r2_score': 0.07267999970304528, 'eval_mean_squared_error': 0.158258855342865, 'eval_accuracy': 0.9180327868852459, 'eval_f1': 0.8331943286071727, 'eval_precision': 0.8484848484848485, 'eval_recall': 0.8209876543209876, 'eval_runtime': 0.1544, 'eval_samples_per_second': 395.046, 'eval_steps_per_second': 25.905, 'epoch': 20.0}
Result[test] {'eval_loss': 0.06254726648330688, 'eval_r2_score': -0.5362268041429159, 'eval_mean_squared_error': 0.2500945031642914, 'eval_accuracy': 0.819672131147541, 'eval_f1': 0.6238095238095238, 'eval_precision': 0.6394557

In [159]:
get_shapley(df, best_trainer, "./ShapleyValuesV3/task3_auth_regressor.pkl");

DATA SIZE 1221


PartitionExplainer explainer: 1222it [32:38,  1.61s/it]


In [160]:
cp ./ShapleyValuesV3/task3_auth_regressor.pkl /content/drive/MyDrive/TalkLikeMom/src/Classifier/ShapleyValuesV3/task3_auth_regressor.pkl

In [163]:
ls /content/drive/MyDrive/TalkLikeMom/src/Classifier/

 data_loader.py                 'Shapley by Lexicon Features.ipynb'
'Evaluate WangChanBERTa.ipynb'   ShapleyValues/
 Lexicons/                       ShapleyValuesV2/
'Lexicons by Shapley.ipynb'      ShapleyValuesV3/
 __pycache__/                   'Train WangChanBERTa 100e.ipynb'
 README.md                      'Train WangChanBERTa.ipynb'
'Run Shapley.ipynb'              utils.py


In [161]:
ls

 data_loader.py                 'Run Shapley.ipynb'
'Evaluate WangChanBERTa.ipynb'  'Shapley by Lexicon Features.ipynb'
 Lexicons/                       ShapleyValues/
'Lexicons by Shapley.ipynb'      ShapleyValuesV3/
 __pycache__/                   'Train WangChanBERTa 100e.ipynb'
 README.md                      'Train WangChanBERTa.ipynb'
 Regressors/                     utils.py


In [164]:
cp -r ./Regressors /content/drive/MyDrive/TalkLikeMom/src/Classifier/RegressorsV3